In [ ]:
! pip install pyrsgis

import os
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose
from keras.constraints import max_norm
from keras import backend as K
import matplotlib.pyplot as plt


# Assign file names
mxDhaka = '/content/drive/Shareddrives/Monami-Extra Space/draft/b1_b9_layerstack.tif'
builtupDhaka = '/content/drive/Shareddrives/Monami-Extra Space/draft/nc_layerstack.tif'
mxLama = '/content/drive/Shareddrives/Monami-Extra Space/draft/b1_b9_layerstack.tif'

# Read the rasters as array
ds1, featuresDhaka = raster.read(mxDhaka, bands='all')
ds2, labelDhaka = raster.read(builtupDhaka, bands='all')
ds3, featuresLama = raster.read(mxLama, bands='all')

# Print the size of the arrays
print("Dhaka Multispectral image shape: ", featuresDhaka.shape)
print("Dhaka Binary built-up image shape: ", labelDhaka.shape)
print("Lama Multispectral image shape: ", featuresLama.shape)

Warning! matplotlib_scalebar library not found. You may not be able to export map directly.


In [ ]:
# Clean the labelled data to replace NoData values by zero
labelDhaka = (labelDhaka == 1).astype(int)

# Reshape the array to single dimensional array
featuresDhaka = changeDimension(featuresDhaka)
labelDhaka = changeDimension (labelDhaka)
featuresLama = changeDimension(featuresLama)
nBands = featuresDhaka.shape[1]

print("Dhaka Multispectral image shape: ", featuresDhaka.shape)
print("Dhaka Binary built-up image shape: ", labelDhaka.shape)
print("Lama Multispectral image shape: ", featuresLama.shape)

Dhaka Multispectral image shape:  (2135280, 9)
Dhaka Binary built-up image shape:  (2135280, 9)
Lama Multispectral image shape:  (2135280, 9)


In [ ]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuresDhaka, labelDhaka, test_size=0.2, random_state=42)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)

In [ ]:
# Normalise the data
xTrain = xTrain / 255.0
xTest = xTest / 255.0
featuresLama = featuresLama / 255.0

# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
featuresLama = featuresLama.reshape((featuresLama.shape[0], 1, featuresLama.shape[1]))

# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, featuresLama.shape)

(1281168, 1, 9) (854112, 1, 9) (2135280, 1, 9)


# Run Model

In [ ]:
# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(1, activation='softmax')],
    )

# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=1)

In [ ]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

predicted = model.predict(featuresLama)
predicted = predicted[:,1]

Confusion matrix: for 14 nodes
 [[134174   2591]
 [  5101  76484]]

P-Score: 0.967, R-Score: 0.937


In [ ]:
# Predict new data and export the probability raster
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outFile = '/content/drive/Shared drives/Monami-Extra Space/Mapathon/DhakaB98.tif'
raster.export(prediction, ds3, filename=outFile, dtype='float')

In [ ]:
import os
import numpy as np

import cv2
from keras.preprocessing.image import img_to_array

In [ ]:
import os 
from PIL import Image
import numpy as np

dirname = '/content/drive/Shareddrives/Monami-Extra Space/DL Reasearch/B9'
files = os.listdir(dirname)
final = []
for fname in files:
    im = Image.open(os.path.join(dirname, fname))
    imarray = np.array(im)
    final.append(imarray)

    

final = np.asarray(final) # shape = (60000,28,28)

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose
from keras.constraints import max_norm
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
final.shape

(20, 43, 62)

In [ ]:
final = final.reshape(final.shape[0], 1, 43, 62)